In [9]:
import torch
import os
import matplotlib.pyplot as plt
if not os.path.exists('.\\data\\'):
    new_directory_path = "..\\..\\"
    os.chdir(new_directory_path)

from src.data.parsers import CocoFreeView
from src.model.rope_positional_embeddings import RopePositionEmbedding

In [14]:
coords = torch.empty((1, 100, 2)).uniform_(-1, 1)

In [16]:
pos = RopePositionEmbedding(
        embed_dim = 128,
        num_heads = 16,
        base = 100,
        min_period = None,
        max_period = None,
        normalize_coords = "separate",
        shift_coords = None,
        jitter_coords = None,
        rescale_coords = None,
        dtype = coords.dtype,
        device = coords.device)
